<a href="https://colab.research.google.com/github/utsavlamichhane/Blood-Glucose-Prediction/blob/main/utsav_final_linear_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Split-Plot Design Analyisis.**


1. MANUAL ANOVA

Lets load the library and the data.

In [1]:
library(readxl); library(dplyr)
H <- c("Toledo","423","465","Mulato","Talismán")
d <- read_excel("spd.xlsx", sheet="Analysis") %>%
  filter(Hy %in% H) %>%
  mutate(
    Block = factor(Block),
    Tr    = factor(Tr, levels=c("Shade","Sun")),
    Hy    = factor(Hy)
  )



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




Ok, now I will cjecj the balnace before going to the Constants and the means.

In [2]:

cat("Obs per Block×Tr:\n"); print(table(d$Block, d$Tr))
cat("\nObs per Hybrid×Tr:\n"); print(table(d$Hy,   d$Tr))


b  <- n_distinct(d$Block)    # 12
a  <- n_distinct(d$Tr)       # 2
s  <- n_distinct(d$Hy)       # 5
N  <- nrow(d)                # 120
gm <- mean(d$Total)
cat("\nGrand mean =", round(gm,2), "\n\n")

Obs per Block×Tr:
   
    Shade Sun
  1    20  20
  2    20  20
  3    20  20

Obs per Hybrid×Tr:
          
           Shade Sun
  423         12  12
  465         12  12
  Mulato      12  12
  Talismán    12  12
  Toledo      12  12

Grand mean = 948.1 



Now, the cell means.

In [3]:

block_means <- d %>% group_by(Block) %>% summarize(bm=mean(Total), .groups="drop")
A_means     <- d %>% group_by(Tr)    %>% summarize(Am=mean(Total), .groups="drop")
B_means     <- d %>% group_by(Hy)    %>% summarize(Bm=mean(Total), .groups="drop")
wp          <- d %>% group_by(Block,Tr) %>% summarize(wpm=mean(Total), .groups="drop")
AB_marg     <- d %>% group_by(Tr,Hy)     %>% summarize(abm=mean(Total), .groups="drop")

Now, I will calculate the folloiwng:
- SS
- Whole-plot error I mean SS(Block: Tr)
- subplot interaction: SS(Tr:Hy)
- residual

In [4]:

SS_Block   <- a * s * sum((block_means$bm - gm)^2)
SS_Tr      <- b * s * sum((A_means$Am    - gm)^2)

wp2        <- wp %>%
  left_join(block_means, by="Block") %>%
  left_join(A_means,       by="Tr")
SS_BlockTr <- s * sum((wp2$wpm - wp2$bm - wp2$Am + gm)^2)
SS_Hy      <- b * a * sum((B_means$Bm    - gm)^2)

AB2        <- AB_marg %>%
  left_join(A_means, by="Tr") %>%
  left_join(B_means, by="Hy")
SS_AB      <- b * sum((AB2$abm - AB2$Am - AB2$Bm + gm)^2)

mod_full   <- lm(Total ~ Block + Tr + Block:Tr + Hy + Tr:Hy, data=d)
SS_Resid   <- sum(resid(mod_full)^2)


cat("SS(Block)      =", round(SS_Block,2), "\n")
cat("SS(Tr)         =", round(SS_Tr,2), "\n")
cat("SS(Block:Tr)   =", round(SS_BlockTr,2), "\n")
cat("SS(Hy)         =", round(SS_Hy,2), "\n")
cat("SS(Tr:Hy)      =", round(SS_AB,2), "\n")
cat("SS(Residual)   =", round(SS_Resid,2), "\n\n")

SS(Block)      = 9217.93 
SS(Tr)         = 933538.2 
SS(Block:Tr)   = 39632.37 
SS(Hy)         = 3660663 
SS(Tr:Hy)      = 839272.6 
SS(Residual)   = 3657543 



Consolidating all of'em.

- Degree of F
- MS
- F
- and the p


In [5]:

df_block  <- b - 1
df_tr     <- a - 1
df_bt     <- df_block
df_hy     <- s - 1
df_ab     <- df_tr * df_hy
df_res    <- N - (df_block+1 + df_tr + df_bt + df_hy + df_ab)

MS_Block  <- SS_Block/df_block
MS_Tr     <- SS_Tr/df_tr
MS_BT     <- SS_BlockTr/df_bt
MS_Hy     <- SS_Hy/df_hy
MS_AB     <- SS_AB/df_ab
MS_Res    <- SS_Resid/df_res

F_Tr      <- MS_Tr/MS_BT
F_Hy      <- MS_Hy/MS_Res
F_AB      <- MS_AB/MS_Res

p_Tr      <- pf(F_Tr, df_tr, df_bt,   lower.tail=FALSE)
p_Hy      <- pf(F_Hy, df_hy, df_res,  lower.tail=FALSE)
p_AB      <- pf(F_AB, df_ab, df_res,  lower.tail=FALSE)

library(tibble)
manual <- tibble(
  Source  = c("Block","Tr",    "Block:Tr","Hy",   "Tr:Hy","Residual"),
  DF      = c(df_block, df_tr, df_bt,     df_hy,  df_ab, df_res),
  SS      = c(SS_Block, SS_Tr, SS_BlockTr,SS_Hy,  SS_AB, SS_Resid),
  MS      = c(MS_Block, MS_Tr, MS_BT,     MS_Hy,  MS_AB, MS_Res),
  F_value = c(NA,       F_Tr,   NA,        F_Hy,   F_AB, NA),
  p_value = c(NA,       p_Tr,   NA,        p_Hy,   p_AB, NA)
)
print(manual, digits=4)


# A tibble: 6 × 6
  Source      DF       SS      MS F_value   p_value
  <chr>    <dbl>    <dbl>   <dbl>   <dbl>     <dbl>
1 Block        2    9218.   4609.   NA    NA       
2 Tr           1  933538. 933538.   47.1   2.06e- 2
3 Block:Tr     2   39632.  19816.   NA    NA       
4 Hy           4 3660663. 915166.   26.5   2.99e-15
5 Tr:Hy        4  839273. 209818.    6.08  1.92e- 4
6 Residual   106 3657543.  34505.   NA    NA       


**Let check the ANOVA from the function.**

And pray the values are same : D

In [19]:
# 1.6 Compare to built‐in aov()
cat("\n--- aov() summary ---\n")
sp.aov <- aov(Total ~ Tr*Hy + Error(Block/Tr), data=d)
summary(sp.aov)




--- aov() summary ---



Error: Block
          Df Sum Sq Mean Sq F value Pr(>F)
Residuals  2  36872   18436               

Error: Block:Tr
          Df  Sum Sq Mean Sq F value Pr(>F)  
Tr         1 3734153 3734153   47.11 0.0206 *
Residuals  2  158529   79265                 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Error: Within
           Df   Sum Sq Mean Sq F value  Pr(>F)    
Hy          4 14642653 3660663  106.09 < 2e-16 ***
Tr:Hy       4  3357090  839273   24.32 2.7e-14 ***
Residuals 106  3657543   34505                    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [17]:
#install.packages("lme4")
#install.packages("emmeans")
#install.packages("car")
#install.packages("glmnet")


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘iterators’, ‘foreach’, ‘shape’




**Now solving the Problem number 2.**

In [13]:

library(lme4); library(emmeans)
mod   <- lmer(Total ~ Tr*Hy + (1|Block) + (1|Block:Tr), data=d)
summary(mod)

emm   <- emmeans(mod, ~ Hy*Tr)
pairs(emm, adjust="tukey")


Welcome to emmeans.
Caution: You lose important information if you filter this package's results.
See '? untidy'

boundary (singular) fit: see help('isSingular')

Warning message in abbreviate(rn, minlength = 11):
“abbreviate used with non-ASCII chars”
Warning message in abbreviate(rn, minlength = 6):
“abbreviate used with non-ASCII chars”
Warning message in abbreviate(rn, minlength = 6):
“abbreviate used with non-ASCII chars”
Warning message in abbreviate(rn, minlength = 6):
“abbreviate used with non-ASCII chars”
Warning message in abbreviate(rn, minlength = 6):
“abbreviate used with non-ASCII chars”
Warning message in abbreviate(rn, minlength = 6):
“abbreviate used with non-ASCII chars”


Linear mixed model fit by REML ['lmerMod']
Formula: Total ~ Tr * Hy + (1 | Block) + (1 | Block:Tr)
   Data: d

REML criterion at convergence: 1487.8

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.6273 -0.5464 -0.1582  0.6229  2.3150 

Random effects:
 Groups   Name        Variance  Std.Dev. 
 Block:Tr (Intercept) 7.173e+02 2.678e+01
 Block    (Intercept) 6.678e-06 2.584e-03
 Residual             3.451e+04 1.858e+02
Number of obs: 120, groups:  Block:Tr, 6; Block, 3

Fixed effects:
                 Estimate Std. Error t value
(Intercept)        592.36      55.81  10.614
TrSun               26.11      78.92   0.331
Hy465             -129.86      75.83  -1.712
HyMulato           484.58      75.83   6.390
HyTalismán         786.25      75.83  10.368
HyToledo          -244.31      75.83  -3.222
TrSun:Hy465        865.83     107.25   8.073
TrSun:HyMulato     472.22     107.25   4.403
TrSun:HyTalismán   -48.33     107.25  -0.451
TrSun:HyToledo     343.75     107.25   3.205

C

Cannot use mode = "kenward-roger" because *pbkrtest* package is not installed

Cannot use mode = "satterthwaite" because *lmerTest* package is not installed



 contrast                      estimate   SE  df z.ratio p.value
 423 Shade - 465 Shade           129.86 75.8 Inf   1.712  0.7888
 423 Shade - Mulato Shade       -484.58 75.8 Inf  -6.390  <.0001
 423 Shade - Talismán Shade     -786.25 75.8 Inf -10.368  <.0001
 423 Shade - Toledo Shade        244.31 75.8 Inf   3.222  0.0419
 423 Shade - 423 Sun             -26.11 78.9 Inf  -0.331  1.0000
 423 Shade - 465 Sun            -762.08 78.9 Inf  -9.656  <.0001
 423 Shade - Mulato Sun         -982.92 78.9 Inf -12.454  <.0001
 423 Shade - Talismán Sun       -764.03 78.9 Inf  -9.681  <.0001
 423 Shade - Toledo Sun         -125.56 78.9 Inf  -1.591  0.8530
 465 Shade - Mulato Shade       -614.44 75.8 Inf  -8.102  <.0001
 465 Shade - Talismán Shade     -916.11 75.8 Inf -12.080  <.0001
 465 Shade - Toledo Shade        114.44 75.8 Inf   1.509  0.8890
 465 Shade - 423 Sun            -155.97 78.9 Inf  -1.976  0.6159
 465 Shade - 465 Sun            -891.94 78.9 Inf -11.301  <.0001
 465 Shade - Mulato Sun  






**Solving the problem number 3**




In [18]:

library(car)
lm0   <- lm(Total ~ Tr*Hy, data=d)
cat("\nVIFs:\n"); print(vif(lm0))


cn <- kappa(cor(model.matrix(lm0)[,-1]), exact=TRUE)
cat("\nCondition number:", round(cn,2), "\n")


library(glmnet)
x <- model.matrix(~ Tr*Hy -1, d); y <- d$Total
cv <- cv.glmnet(x,y,alpha=0)
print(coef(glmnet(x,y,alpha=0,lambda=cv$lambda.min)))

contrasts(d$Hy) <- contr.helmert(5)
contrasts(d$Tr) <- contr.helmert(2)
summary(lm(Total~Tr*Hy, data=d))

mm2 <- lmer(Total ~ Tr*Hy + (1+Tr|Block), data=d)
Anova(mm2, type=3)





VIFs:


there are higher-order terms (interactions) in this model
consider setting type = 'predictor'; see ?vif



      GVIF Df GVIF^(1/(2*Df))
Tr       5  1        2.236068
Hy      16  4        1.414214
Tr:Hy   48  4        1.622390

Condition number: 37 


Loaded glmnet 4.1-8



11 x 1 sparse Matrix of class "dgCMatrix"
                         s0
(Intercept)       633.37484
TrShade           -32.51962
TrSun              38.49672
Hy465            -100.06457
HyMulato          460.23253
HyTalismán        723.71923
HyToledo         -225.72004
TrSun:Hy465       751.85136
TrSun:HyMulato    425.73752
TrSun:HyTalismán  -37.67859
TrSun:HyToledo    261.09372



Call:
lm(formula = Total ~ Tr * Hy, data = d)

Residuals:
    Min      1Q  Median      3Q     Max 
-494.44  -99.06  -26.88  112.22  434.72 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  948.097     17.085  55.494  < 2e-16 ***
Tr1          176.403     17.085  10.325  < 2e-16 ***
Hy1          151.528     27.013   5.609 1.53e-07 ***
Hy2          189.722     15.596  12.165  < 2e-16 ***
Hy3          105.208     11.028   9.540 4.44e-16 ***
Hy4         -103.778      8.542 -12.149  < 2e-16 ***
Tr1:Hy1      216.458     27.013   8.013 1.28e-12 ***
Tr1:Hy2        6.551     15.596   0.420    0.675    
Tr1:Hy3      -61.794     11.028  -5.603 1.57e-07 ***
Tr1:Hy4        2.132      8.542   0.250    0.803    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 187.2 on 110 degrees of freedom
Multiple R-squared:  0.8494,	Adjusted R-squared:  0.8371 
F-statistic: 68.94 on 9 and 110 DF,  p-value: < 2.2e-16


boundary (singular) fit: see help('isSingular')



,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>
(Intercept),3153.29901,1,0.000000e+00
Tr,47.10927,1,6.713737e-12
Hy,428.05337,4,2.409262e-91
Tr:Hy,98.13890,4,2.448957e-20


Justification & Choice

- Severity: The original design shows only mild multicollinearity (VIF ≤ 2.24, condition number = 37), which does not threaten the validity of our hypothesis tests or confidence intervals.

- Ridge regression is a useful safety check—its near‐identical coefficients reassure us—but it introduces bias and complicates interpretation.

- Helmert contrasts perfectly eliminate correlation among predictors, which can be valuable for publication tables or teaching contrasts, but they change the parameterization.

**Conclusion**: Because collinearity is mild, we retain the straightforward Tr×Hy parameterization for primary inference. If orthogonal parameter estimates are ever needed (e.g. for clearer coefficient tables), switching to Helmert contrasts is the simplest remedy.